### Essaymaker Agent flow

#### Initialize variables

In [4]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv('/Users/felipe_q/Desktop/AI_models/Agents/agent_venv/Agent_Development/langraph/credentials.env'))

True

In [14]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage

memory = SqliteSaver.from_conn_string(":memory:")

### Initialize Agent class